In [1]:
#Imports
import numpy as np
from midiutil import MIDIFile
import matplotlib.pyplot as plt
import ternary
from ternary.helpers import simplex_iterator
import graphviz
import anytree as at
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
import theano.tensor as T
from theano import function

# helpers
import sys
sys.path.insert(0, 'helpers/')
from tree_parser import TreeParser
from midi import *

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Spouep\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# CONSTANTS
L = 16 # length of measure
N = int(np.log2(L)+1) # Number of levels (including 0)

In [48]:
# Initial THEANO symbolic variables

p1 = T.dscalar('p1')
p2 = T.dscalar('p2')
p3 = T.dscalar('p3')

In [4]:
# 2-ways dictionary, not too slow...
class bidict(dict):
    def __init__(self, *args, **kwargs):
        super(bidict, self).__init__(*args, **kwargs)
        self.inverse = {}
        for key, value in self.items():
            self.inverse.setdefault(value,[]).append(key) 

    def __setitem__(self, key, value):
        if key in self:
            self.inverse[self[key]].remove(key) 
        super(bidict, self).__setitem__(key, value)
        self.inverse.setdefault(value,[]).append(key)        

    def __delitem__(self, key):
        self.inverse.setdefault(self[key],[]).remove(key)
        if self[key] in self.inverse and not self.inverse[self[key]]: 
            del self.inverse[self[key]]
        super(bidict, self).__delitem__(key)

# Import stored pickle files

In [35]:
import pickle
from six.moves import cPickle

all_trees = cPickle.load( open( "files/all_trees.pickle", "rb" ) )
all_rhythms = pickle.load( open( "files/all_rhythms.pickle", "rb" ) )
tree2rhythm = pickle.load( open( "files/tree2rhythm.pickle", "rb" ) )
rhythm2trees = pickle.load( open( "files/rhythm2trees.pickle", "rb" ) )
#tree2prob = pickle.load( open( "files/tree2prob.pickle", "rb" ) )

# Set of all trees

In [50]:
# Templates level 1
templates_1 = set()
templates_1.add(p2)
templates_1.add(p3)

# Templates level 2
templates_2 = set()
templates_2.add(p2)
templates_2.add(p3)

for elem1 in templates_1:
    for elem2 in templates_1:
        templates_2.add((elem1, elem2))
        
# Templates level 3
templates_3 = set()
templates_3.add(p2)
templates_3.add(p3)

for elem1 in templates_2:
    for elem2 in templates_2:
        templates_3.add((elem1, elem2))
        
# Templates level 4
templates_4 = set()
templates_4.add(p2)
templates_4.add(p3)

for elem1 in templates_3:
    for elem2 in templates_3:
        templates_4.add((elem1, elem2))
        
# Templates level 5
all_trees = set()
all_trees.add(p2)
all_trees.add(p3)

for elem1 in templates_4:
    for elem2 in templates_4:
        all_trees.add((elem1, elem2))
        
#save

    
from six.moves import cPickle
with open('files/all_trees.pickle', 'wb') as f:
    cPickle.dump(all_trees, f, protocol=cPickle.HIGHEST_PROTOCOL)

len(all_trees)

2090918

# Set of all rhythms

In [8]:
all_rhythms = set()

for i in range(2**16): # populating
    all_rhythms.add( tuple(("{0:0%sb}" %L).format(i)) )
    
#save
import pickle
with open('files/all_rhythms.pickle', 'wb') as f:
    pickle.dump(all_rhythms, f)
    
len(all_rhythms)

65536

# 2-way dictionnary between sets of trees and rhythms

In [49]:
def tree_to_rhythm(tree, size=L):    
    if(p2 == tree):
        tup = ('1',) + ('0',) * int(size-1)
        return tup    
    elif(p3 == tree):
        tup = ('0',) * int(size)
        return tup    
    else:
        return tree_to_rhythm(tree[0], size/2) + tree_to_rhythm(tree[1], size/2)
    
trees2rhythms = bidict() # initial bidirection dictionary (can't be stored with pickle.....)

for tree in all_trees:
    trees2rhythms[tree] = tree_to_rhythm(tree)
    
# 2 dict for the 2 directions 
rhythm2trees = trees2rhythms.inverse
tree2rhythm  = dict(trees2rhythms)

#save
import pickle
with open('files/tree2rhythm.pickle', 'wb') as f:
    pickle.dump(tree2rhythm, f)
with open('files/rhythm2trees.pickle', 'wb') as f:
    pickle.dump(rhythm2trees, f)

IndexError: too many indices for array

# Probability of each tree (set)

In [ ]:
def tree_to_prob(tree) -> T.TensorVariable :    
    if(p2 == tree):
        return p2 
    elif(p3 == tree):
        return p3
    else:
        return p1 * tree_to_prob(tree[0]) * tree_to_prob(tree[1])    

tree2prob = {}
i=0

for tree in all_trees:
    i+=1
    if(i%1000 == 0):
        print(i)
        
    tree2prob[tree] = tree_to_prob(tree)
    
#save
import pickle
with open('files/tree2prob.pickle', 'wb') as f:
    pickle.dump(tree2prob, f)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


# Probability of each rhythm (set)

In [ ]:
rhythm2prob = {}

for rhythm in all_rhythms: 
    prob_rhythm = 0
    for tree in rhythm2trees[rhythm]:
        prob_rhythm += tree2prob[tree]
        
    rhythm2prob[rhythm] = prob_rhythm

#save
import pickle
with open('files/rhythm2prob.pickle', 'wb') as f:
    pickle.dump(rhythm2prob, f)

# Set of all templates (?)

In [62]:
# Templates level 1
templates_1 = set()
templates_1.add(p2)
templates_1.add(p3)
templates_1.add(tuple())

# Templates level 2
templates_2 = set()
templates_2.add(p2)
templates_2.add(p3)

for elem1 in templates_1:
    for elem2 in templates_1:
        templates_2.add((elem1, elem2))
        
# Templates level 3
all_templates = set()
all_templates.add(p2)
all_templates.add(p3)

for elem1 in templates_2:
    for elem2 in templates_2:
        all_templates.add((elem1, elem2))
               
#save 
from six.moves import cPickle
with open('files/all_templates.pickle', 'wb') as f:
    cPickle.dump(all_templates, f, protocol=cPickle.HIGHEST_PROTOCOL)

len(all_templates)

123

# 2-ways dict between templates and trees

In [55]:
e

((((p3, p2), (p2, p3)), ((p2, p2), p2)), (((p2, p2), p3), (p3, (p2, p3))))

In [58]:
lo = ()

tuple